In [3]:
# Handling Data for keras 
from __future__ import print_function
import os
import sys
import numpy as np
np.random.seed(1337)

import pandas as pd
import random
#from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, Convolution1D, MaxPooling1D, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [4]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [5]:
df = pd.read_csv('./data/data.csv')

In [6]:
# Removing NaNs (only 5 rows)
df = df[pd.notnull(df['title'])]

pos = df[df['label'] == 1]
neg = df[df['label'] == 0]

In [7]:
# We have more neg samples than pos, so even things out
neg_sample = neg.sample(len(pos))

# Merging pos and neg
data = pos.append(neg_sample, ignore_index=True)

# Get Train, Test and Valid data ready
X, Y = np.array(data['title']), np.array(data['label'])

In [8]:
BASE_DIR = './data/'
GLOVE_DIR = BASE_DIR + '/glove.6B/'

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

Indexing word vectors.


In [9]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [10]:
print('Found %s word vectors.' % len(embeddings_index))

#Passing through tokenizer
texts = X.tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 400000 word vectors.
Found 41021 unique tokens.


In [11]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np_utils.to_categorical(np.asarray(Y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.1 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of data tensor: (96058, 1000)
Shape of label tensor: (96058, 2)


In [12]:
nb_words = len(word_index)
embedding_layer = Embedding(nb_words + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [13]:
model = Sequential()

model.add(embedding_layer)
model.add(Convolution1D(128, 5))
model.add(Activation('relu'))
model.add(MaxPooling1D(5))

model.add(Convolution1D(128, 5))
model.add(Activation('relu'))
model.add(MaxPooling1D(5))

model.add(Convolution1D(128, 5))
model.add(Activation('relu'))
model.add(MaxPooling1D(35))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [15]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=1, batch_size=128)

Train on 86453 samples, validate on 9605 samples
Epoch 1/1
 1408/86453 [..............................] - ETA: 1525s - loss: 0.6934 - acc: 0.4908

KeyboardInterrupt: 

In [1]:
from keras.models import load_model
model = load_model('models/cnn1d.hdf5')

Using TensorFlow backend.


ValueError: You are trying to load a weight file containing 6 layers into a model with 5 layers.